# Nginx HTTP框架

## HTTP框架的初始化

### HTTP请求的11个处理阶段

`src/http/ngx_http_core_module.h`

```c
typedef enum
{
    // 接受到完整的HTTP请求头处理的HTTP阶段
    NGX_HTTP_POST_READ_PHASE = 0,

    // 请求的URI和Location表达式匹配前，修改请求的URI
    NGX_HTTP_SERVER_REWRITE_PHASE,

    // 根据请求的URI匹配Location表达式
    NGX_HTTP_FIND_CONFIG_PHASE,

    // 寻找匹配Location后，再修改请求的URI
    NGX_HTTP_REWRITE_PHASE,

    // 这个阶段用于rewrite重写URI后，防止错误的nginx.conf导致死循环
    NGX_HTTP_POST_REWRITE_PHASE,

    NGX_HTTP_PREACCESS_PHASE,
    NGX_HTTP_ACCESS_PHASE,
    NGX_HTTP_POST_ACCESS_PHASE,

    NGX_HTTP_TRY_FILES_PHASE,

    // 处理HTTP请求内容阶段
    NGX_HTTP_CONTENT_PHASE,

    // 处理完请求记录日志阶段
    NGX_HTTP_LOG_PHASEs
} ngx_http_phases;
```

## HTTP框架的执行流程

在请求的生命周期内，基于事件驱动的HTTP框架是怎么处理网络事件，以及怎么集成HTTP模块功能处理HTTP请求。

HTTP框架存在的主要目的：

* Nginx的事件框架主要针对的是传输层TCP，作为Web服务器需要处理的HTTP，HTTP框架针对基于TCP的事件框架解决好HTTP的网络传输、解析和组装等问题
* 事件驱动框架性能是很好，但开发效率不高，HTTP业务通常比较复杂，我们希望HTTP模块拥有事件框架高性能的同时，尽量只关注业务。HTTP框架为HTTP模块屏蔽事件驱动架构，使得HTTP模块不需要关系网络事件的处理，同时又能灵活介入11阶段以处理请求

HTTP框架处理的流程：

* 与客户端建立TCP连接
* 接收、解析HTTP请求头行和头
* 根据`nginx.conf`配置，找到对应HTTP模块依次处理

对于TCP网络事件，可以粗略的分为读事件和写事件，读事件可以细粉为收到SYN包的新的连接事件、收到FIN包的连接关闭事件，以及套接字缓冲区接收的TCP流。